In [1]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, regularizers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import pathlib

In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [17]:
import os
import numpy as np
data_dir = 'C:/Users/hecr/.keras/datasets/flower_photos'

# Load all file paths and labels
image_paths = []
labels = []
class_names = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])

for label, class_name in enumerate(class_names):
    class_dir = os.path.join(data_dir, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        if os.path.isfile(img_path):
            image_paths.append(img_path)
            labels.append(label)

# Convert lists to numpy arrays for splitting
image_paths = np.array(image_paths)
labels = np.array(labels)

# Split data into training (80%), validation (10%), and test (10%) sets
train_paths, temp_paths, train_labels, temp_labels = train_test_split(image_paths, labels, test_size=0.3, stratify=labels, random_state=42)
val_paths, test_paths, val_labels, test_labels = train_test_split(temp_paths, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)

# Helper function to create TensorFlow datasets
def create_dataset(image_paths, labels, batch_size=16):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    def load_image(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (224, 224))
        return img, label

    dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

# Create datasets
train_dataset = create_dataset(train_paths, train_labels)
val_dataset = create_dataset(val_paths, val_labels)
test_dataset = create_dataset(test_paths, test_labels)

# Optionally print the sizes
print(f'Train set size: {len(train_paths)}')
print(f'Validation set size: {len(val_paths)}')
print(f'Test set size: {len(test_paths)}')

Train set size: 2569
Validation set size: 550
Test set size: 551


In [5]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.Rescaling(1./255),
    layers.RandomFlip("horizontal",
                      input_shape=(224,
                                  224,
                                  3)),
    layers.RandomRotation(0.1),
  ]
)
model = Sequential([
  data_augmentation,
  layers.Conv2D(32, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
  layers.Conv2D(32, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
  layers.Conv2D(32, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
  layers.Flatten(),
  layers.Dense(64, activation='relu', activity_regularizer=regularizers.L2(0.01)),
  layers.Dropout(0.1),
  layers.Dense(5, name="outputs", activation="sigmoid")
])

In [6]:
optimizerr = optimizers.Adam(learning_rate=0.005)

# Compile the model with the optimizer object
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset,
                    epochs=20, 
                    validation_data=val_dataset,
                    )

Epoch 1/20
184/184 [==============================] - 105s 562ms/step - loss: 1.3208 - accuracy: 0.4768 - val_loss: 1.2488 - val_accuracy: 0.5504
Epoch 2/20
184/184 [==============================] - 106s 574ms/step - loss: 1.1151 - accuracy: 0.5916 - val_loss: 1.1675 - val_accuracy: 0.5531
Epoch 3/20
184/184 [==============================] - 106s 575ms/step - loss: 1.0103 - accuracy: 0.6434 - val_loss: 1.0315 - val_accuracy: 0.6349
Epoch 4/20
184/184 [==============================] - 108s 583ms/step - loss: 0.9323 - accuracy: 0.6744 - val_loss: 0.9540 - val_accuracy: 0.6649
Epoch 5/20
184/184 [==============================] - 103s 557ms/step - loss: 0.8774 - accuracy: 0.7027 - val_loss: 0.9020 - val_accuracy: 0.7084
Epoch 6/20
184/184 [==============================] - 103s 558ms/step - loss: 0.8398 - accuracy: 0.7119 - val_loss: 0.8731 - val_accuracy: 0.7193
Epoch 7/20
184/184 [==============================] - 102s 555ms/step - loss: 0.8006 - accuracy: 0.7316 - val_loss: 0.8573 -

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)

# Print the test loss and accuracy
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

23/23 [==============================] - 2s 67ms/step - loss: 0.7168 - accuracy: 0.7575
Test Loss: 0.7167800068855286
Test Accuracy: 0.7574931979179382


In [24]:
model1 = Sequential([
  data_augmentation,
  layers.Conv2D(16, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
  layers.Conv2D(32, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D((2, 2)),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D((5, 5)),
  layers.Flatten(),
  layers.Dense(64, activation='relu', activity_regularizer=regularizers.L1(0.01)),
  layers.Dropout(0.1),
  layers.Dense(64, activation='tanh'),
  layers.Dense(5, name="outputs", activation="softmax")
])

In [25]:
model1.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
history = model1.fit(train_dataset,
                    epochs=10, 
                    validation_data=val_dataset,
                    )

Epoch 1/10
161/161 [==============================] - 69s 422ms/step - loss: 1.4325 - accuracy: 0.3671 - val_loss: 1.2770 - val_accuracy: 0.4745
Epoch 2/10
161/161 [==============================] - 74s 458ms/step - loss: 1.1563 - accuracy: 0.5434 - val_loss: 1.1043 - val_accuracy: 0.5600
Epoch 3/10
161/161 [==============================] - 74s 460ms/step - loss: 1.0447 - accuracy: 0.6061 - val_loss: 0.9479 - val_accuracy: 0.6455
Epoch 4/10
161/161 [==============================] - 76s 469ms/step - loss: 0.9511 - accuracy: 0.6469 - val_loss: 0.9370 - val_accuracy: 0.6600
Epoch 5/10
161/161 [==============================] - 74s 460ms/step - loss: 0.9069 - accuracy: 0.6734 - val_loss: 0.9899 - val_accuracy: 0.6382
Epoch 6/10
161/161 [==============================] - 73s 454ms/step - loss: 0.8714 - accuracy: 0.6878 - val_loss: 0.8943 - val_accuracy: 0.6800
Epoch 7/10
161/161 [==============================] - 75s 462ms/step - loss: 0.8388 - accuracy: 0.6851 - val_loss: 0.8284 - val_ac

In [27]:
test_loss, test_accuracy = model1.evaluate(test_dataset)

# Print the test loss and accuracy
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

35/35 [==============================] - 2s 53ms/step - loss: 0.8205 - accuracy: 0.7042
Test Loss: 0.8205288648605347
Test Accuracy: 0.7041742205619812
